In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np
import pandas as pd

In [2]:
link_list = list(pd.read_csv('tthc_link.csv')['TTHCLienQuan'].values)

In [3]:
driver = webdriver.Chrome()

cols_list = ['link', 
             'maThuTuc',
             'soQuyetDinh',
             'tenThuTuc',
             'capThucHien',
             'loaiThuTuc',
             'linhVuc',
             'trinhTuThucHien',
             'cachThucThucHien',
             'thanhPhanHoSo',
             'doiTuongThucHien',
             'coQuanThucHien',
             'coQuanCoThamQuyen',
             'diaChiTiepNhanHoSo',
             'coQuanDuocUyQuyen',
             'coQuanPhoiHop',
             'ketQuaThucHien',
             'canCuPhapLy',
             'yeuCauDieuKienThucHien',
             'tuKhoa',
             'moTa']

try:
    df = pd.read_csv('tthc_detail.csv', dtype={'maThuTuc': str})
    if df.empty:
        print("CSV file is empty")
        # Optionally create an empty DataFrame with your expected columns
        df = pd.DataFrame(columns=cols_list)
except pd.errors.EmptyDataError:
    print("File is completely empty")
    df = pd.DataFrame(columns=cols_list)
    df.to_csv('tthc_detail.csv', index=False)

for i in range(len(link_list)):
    # Go to detail page link of each question
    if link_list[i] in list(df['link'].values):
        continue

    driver.get(link_list[i])

    wait = WebDriverWait(driver, 5)

    # Locate the "Xem chi tiết" link
    try:
        link = driver.find_element(By.LINK_TEXT, "Xem chi tiết")
    except:
        df.loc[i, cols_list] = [link_list[i]] + [''] * 20
        continue        

    # Extract href and navigate to it
    href = link.get_attribute("href")
    driver.get(href)
    
    # Find all matching elements
    try:
        modal_content = wait.until(
            EC.visibility_of_element_located((By.CLASS_NAME, "modal-content"))
        )
    except:
        df.loc[i, cols_list] = [link_list[i]] + [''] * 20
        continue
        
    while True:
        if str(modal_content.find_element(By.CLASS_NAME, "col-sm-9.col-xs-12").text) == link_list[i].split('=')[-1]:
            break
        if str(modal_content.find_element(By.CLASS_NAME, "col-sm-9.col-xs-12").text) != '':
            break
    elements = modal_content.find_elements(By.CLASS_NAME, "col-sm-9.col-xs-12")

    # Extract text content into a list
    results = []
    results.append(link_list[i])
    for element in elements:
        try:
            results.append(str(element.text).strip())
        except:
            results.append('')

    if len(results) != 21:
        raise link_list[i]

    df.loc[i, cols_list] = results
    df.to_csv('tthc_detail.csv', index=False)   
    
    print(f'Done of {len(link_list)}: {i+1}', end='\r')
  
driver.quit()  

In [4]:
df = pd.read_csv('tthc_detail.csv', dtype={'maThuTuc': str})
df.to_json('tthc_detail.json', force_ascii=False)
df.tail()

,link,maThuTuc,soQuyetDinh,tenThuTuc,capThucHien,loaiThuTuc,linhVuc,trinhTuThucHien,cachThucThucHien,thanhPhanHoSo,...,coQuanThucHien,coQuanCoThamQuyen,diaChiTiepNhanHoSo,coQuanDuocUyQuyen,coQuanPhoiHop,ketQuaThucHien,canCuPhapLy,yeuCauDieuKienThucHien,tuKhoa,moTa
2699,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,1.004356,87/QĐ-BTNMT,"Kiểm tra, xác nhận hoàn thành công trình bảo v...",Cấp Tỉnh,TTHC được luật giao quy định chi tiết,Môi trường,Bước 1. Nộp hồ sơ: Chủ dự án nộp hồ sơ báo cáo...,"Hình thức nộp Thời hạn giải quyết Phí, lệ phí ...","Bao gồm\nTên giấy tờ Mẫu đơn, tờ khai Số lượng...",...,Sở Tài nguyên và Môi trường,Ủy ban nhân dân cấp Tỉnh,Chủ dự án nộp hồ sơ báo cáo kết quả thực hiện ...,Không có thông tin,Không có thông tin,Giấy xác nhận hoàn thành công trình bảo vệ môi...,Số ký hiệu Trích yếu Ngày ban hành Cơ quan ban...,Không có thông tin,Không có thông tin,Không có thông tin
2700,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,2.001825,87/QĐ-BTNMT,"Kiểm tra, xác nhận hoàn thành công trình bảo v...",Cấp Bộ,TTHC không được luật giao cho địa phương quy đ...,Môi trường,a) Trình tự thực hiện\nBước 1. Nộp hồ sơ: Chủ ...,"Hình thức nộp Thời hạn giải quyết Phí, lệ phí ...","Bao gồm\nTên giấy tờ Mẫu đơn, tờ khai Số lượng...",...,"Cơ quan chuyên môn được Bộ, cơ quan ngang Bộ giao","Bộ, cơ quan ngang Bộ, cơ quan thuộc Chính phủ",Văn phòng tiếp nhận và trả kết quả giải quyết ...,Không có thông tin,Không có thông tin,Giấy xác nhận hoàn thành công trình bảo vệ môi...,Số ký hiệu Trích yếu Ngày ban hành Cơ quan ban...,không quy định,Không có thông tin,Không có thông tin
2701,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,1.000935,1792 /QĐ-BKHCN,"Thủ tục đánh giá, nghiệm thu và công nhận kết ...",Cấp Bộ,TTHC được luật giao quy định chi tiết,Sở hữu trí tuệ,- Bước 1: Tổ chức chủ trì và chủ nhiệm dự án t...,"Hình thức nộp Thời hạn giải quyết Phí, lệ phí ...","Bao gồm\nTên giấy tờ Mẫu đơn, tờ khai Số lượng...",...,Cục Sở hữu trí tuệ - Bộ Khoa học và Công nghệ,Bộ Khoa học và Công nghệ,Không có thông tin,Không có thông tin,Không có thông tin,Quyết định công nhận kết quả thực hiện dự án T...,Số ký hiệu Trích yếu Ngày ban hành Cơ quan ban...,Không có thông tin,"đánh giá, nghiệm thu và công nhận kết quả thực...",Không có thông tin
2702,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,2.000457,1792 /QĐ-BKHCN,Thủ tục chấm dứt hợp đồng trong quá trình thực...,Cấp Bộ,TTHC không được luật giao cho địa phương quy đ...,Sở hữu trí tuệ,"Bước 1: Tổ chức chủ trì, chủ nhiệm dự án nộp H...","Hình thức nộp Thời hạn giải quyết Phí, lệ phí ...","Bao gồm\nTên giấy tờ Mẫu đơn, tờ khai Số lượng...",...,Cục Sở hữu trí tuệ - Bộ Khoa học và Công nghệ,Bộ Khoa học và Công nghệ,Không có thông tin,Không có thông tin,Không có thông tin,Quyết định chấm dứt hợp đồng thực hiện dự án T...,Số ký hiệu Trích yếu Ngày ban hành Cơ quan ban...,- Có căn cứ để khẳng định việc thực hiện hoặc ...,chấm dứt hợp đồng trong quá trình thực hiện dự...,Không có thông tin
2703,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,2.000895,1792 /QĐ-BKHCN,"Thủ tục đánh giá, nghiệm thu và công nhận kết ...",Cấp Tỉnh,TTHC không được luật giao cho địa phương quy đ...,Sở hữu trí tuệ,- Bước 1: Tổ chức chủ trì và chủ nhiệm dự án t...,"Hình thức nộp Thời hạn giải quyết Phí, lệ phí ...","Bao gồm\nTên giấy tờ Mẫu đơn, tờ khai Số lượng...",...,Sở Khoa học và Công nghệ,Không có thông tin,Sở Khoa học và Công nghệ,Không có thông tin,Không có thông tin,Quyết định công nhận kết quả thực hiện dự án c...,Số ký hiệu Trích yếu Ngày ban hành Cơ quan ban...,Không,Không có thông tin,Không có thông tin
